In [1]:
import transformers
import torch
from transformers import pipeline
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device
    
model = transformers.T5ForConditionalGeneration.from_pretrained("google/flan-ul2", torch_dtype=torch.bfloat16,  cache_dir="/mnt/disk-1/flan-ul2")         
tokenizer = transformers.AutoTokenizer.from_pretrained("google/flan-ul2")
        
model, tokenizer = accelerator.prepare(model, tokenizer)

/home/yuxizheng/miniconda3/envs/get_responses/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:28<00:00,  3.58s/it]


In [4]:
batches = ["Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel?",
"Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil?",
"Answer using only one word - Boa python or Snake and not Nail. Which is more similar in meaning to Nail?",
"Answer using only one word - Chisel or Toad and not Paint brush. Which is more similar in meaning to Paint brush?",
"Answer using only one word - Caiman or Crocodile and not Shovel. Which is more similar in meaning to Shovel?"]



inputs = tokenizer(batches, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_length=512, do_sample=False)

responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


prompt_and_response = list(zip(batches, responses))

# prompt_and_response = []
# for prompt, response in zip(batches, responses):
#     prompt_and_response.append([prompt, response])
    
prompt_and_response

[('Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel?',
  'Spanner'),
 ('Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil?',
  'Tortoise'),
 ('Answer using only one word - Boa python or Snake and not Nail. Which is more similar in meaning to Nail?',
  'Snake'),
 ('Answer using only one word - Chisel or Toad and not Paint brush. Which is more similar in meaning to Paint brush?',
  'Toad'),
 ('Answer using only one word - Caiman or Crocodile and not Shovel. Which is more similar in meaning to Shovel?',
  'Crocodile')]

In [ ]:
import csv
def save_responses(reponses, file_path):
    """the helper function for saving the responses"""
    
    with open(file_path, 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerows(reponses)
        
save_responses(prompt_and_response, "test.csv")